In [1]:
from keras.models import Sequential, model_from_json
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.constraints import maxnorm
from keras import regularizers
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.sequence import pad_sequences

from keras.layers import *
from keras.models import *

from keras.regularizers import l2

from keras.applications import imagenet_utils, Xception
from keras.preprocessing.image import img_to_array
import keras.backend as K

import numpy as np 
import json
import os
import cv2
import h5py
import pickle

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

In [3]:
from helpers import TrainingMonitor
from helpers import Utils

In [4]:
Xoh_file = open('../input/dataset/Xoh.pickle', 'rb')
x = pickle.load(Xoh_file)
Xoh_file.close()

In [5]:
Yoh_file = open('../input/dataset/Yoh.pickle', 'rb')
y = pickle.load(Yoh_file)
Yoh_file.close()

In [28]:
y = pad_sequences(y, maxlen=11, padding="post", value=[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [6]:
x[0].shape

(30, 37)

In [7]:
y[0]

array([[ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.]])

In [8]:
y_input = np.zeros([1000, 10, 11])
y_input[:, 1:] = y[:,:-1]

In [9]:
y_input[0]

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [10]:
train_split, val_split = 0.7, 0.15

train_len = int(train_split * x.shape[0])
train_val_len = int((train_split + val_split) * x.shape[0])

x_train = x[:train_len]
x_val = x[train_len:train_val_len]
x_test = x[train_val_len:]

y_input_train = y_input[:train_len]
y_input_val = y_input[train_len:train_val_len]
y_input_test = y_input[train_val_len:]

y_train = y[:train_len]
y_val = y[train_len:train_val_len]
y_test = y[train_val_len:]

In [11]:
x_train.shape

(700, 30, 37)

In [12]:
y_train.shape

(700, 10, 11)

In [13]:
INPUT_LENGTH = TIME_STEPS = x_train.shape[1] 
INPUT_DIM = x_train.shape[2]
OUTPUT_LENGTH = y_train.shape[1]

In [14]:
encoder_input = Input(shape=(INPUT_LENGTH,))
decoder_input = Input(shape=(OUTPUT_LENGTH,))

In [63]:
def attention(inputs):
    input_dim = int(inputs.shape[2])
    a = Permute((2, 1))(inputs)
    a = Dense(TIME_STEPS, activation='softmax')(a)
    a_probs = Permute((2, 1))(a)
    output_attention_mul = multiply([inputs, a_probs])
    return output_attention_mul

In [23]:
from keras import layers 
from keras import Input 
from keras.models import Model 

left_input = Input(shape=(30, 37))
left_output = LSTM(128, return_sequences=False)(left_input)
left_output = RepeatVector(10)(left_output)
left_output = LSTM(64, return_sequences=True)(left_output)

right_input = Input(shape=(10, 11)) 
right_output = LSTM(64, return_sequences=True)(right_input)

merged = layers.concatenate([left_output, right_output], axis=-1) 

output = TimeDistributed(Dense(11, activation="softmax"))(merged) 
model = Model([left_input, right_input], output)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           (None, 30, 37)       0                                            
__________________________________________________________________________________________________
lstm_18 (LSTM)                  (None, 128)          84992       input_13[0][0]                   
__________________________________________________________________________________________________
repeat_vector_7 (RepeatVector)  (None, 10, 128)      0           lstm_18[0][0]                    
__________________________________________________________________________________________________
input_14 (InputLayer)           (None, 10, 11)       0                                            
__________________________________________________________________________________________________
lstm_19 (L

In [24]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"]) 

In [25]:
output_path = "../output/"

print(os.getpid())
filepath=output_path + "progress/seq2seq-mulitiheads-weights-best.hdf5"
MC = ModelCheckpoint(filepath, monitor='val_acc', verbose=0, save_best_only=True, mode='max')

figPath = os.path.sep.join([output_path, "monitor/{}.png".format(os.getpid())])
jsonPath = os.path.sep.join([output_path, "monitor/{}.json".format(os.getpid())])
TM = TrainingMonitor(figPath, jsonPath=jsonPath, startAt=0)

RLR = ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0, patience=5, min_lr=0.5e-6)

callbacks = [MC, TM, RLR]

2464


In [26]:
history = model.fit([x_train, y_input_train], y_train,
            batch_size=32,
            epochs=100,
            validation_data=([x_val, y_input_val], y_val),
            shuffle="batch",
            callbacks=callbacks)

Train on 700 samples, validate on 150 samples
Epoch 1/100
700/700 [==============================] - 6s 8ms/step - loss: 2.2363 - acc: 0.2320 - val_loss: 2.1314 - val_acc: 0.3000
Epoch 2/100
700/700 [==============================] - 2s 2ms/step - loss: 2.0864 - acc: 0.2943 - val_loss: 2.0341 - val_acc: 0.3347
Epoch 3/100
700/700 [==============================] - 2s 3ms/step - loss: 1.9675 - acc: 0.3567 - val_loss: 1.8855 - val_acc: 0.3600
Epoch 4/100
700/700 [==============================] - 2s 3ms/step - loss: 1.8266 - acc: 0.4074 - val_loss: 1.7467 - val_acc: 0.4187
Epoch 5/100
700/700 [==============================] - 2s 2ms/step - loss: 1.6263 - acc: 0.4740 - val_loss: 1.4342 - val_acc: 0.4720
Epoch 6/100
700/700 [==============================] - 2s 2ms/step - loss: 1.3627 - acc: 0.4939 - val_loss: 1.3007 - val_acc: 0.4960
Epoch 7/100
700/700 [==============================] - 2s 3ms/step - loss: 1.2498 - acc: 0.5341 - val_loss: 1.2125 - val_acc: 0.5473
Epoch 8/100
700/700 [==

In [27]:
scores = model.evaluate([x_test, y_input_test], y_test, verbose=0)
print("Train: %.2f%%; Val: %.2f%%; Test: %.2f%%" % 
      (np.max(history.history['acc'])*100, np.max(history.history['val_acc'])*100, scores[1]*100)
     )

Train: 78.70%; Val: 76.47%; Test: 76.87%


In [118]:
model_json = model.to_json()
with open("../output/saved/date_model_seq2seq_multiheads_80.20.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("../output/saved/date_weight_seq2seq_attention_80.20.hdf5")